In [98]:
import numpy as np
import scipy.sparse as sparse
import scipy.linalg as la

def grad(f, x):
    '''
        Input:
            f: lambda function
            x: function args
        Output:
            grad_f: function gradient at x
    '''
    n = len(x)
    grad_f = np.zeros(n)
    
    E = np.diag([pow(np.finfo(float).eps, 1/3) * (abs(a) + 1) for a in x])
    
    for i in range(n):
        grad_f[i] = (f(x + E[:, i]) - f(x - E[:, i])) * (0.5 / E[i, i])
    
    return grad_f
    
def hess(f, x):
    '''
        Input:
            f: lambda function
            x: function args
        Output:
            hess_f: hessian of f at x
    '''
    n = len(x)
    hess_f = np.zeros([n, n])
    
    E = np.diag([pow(np.finfo(float).eps, 1/4) * (abs(a) + 1) for a in x])
    
    for i in range(n):
        for j in range(n):
            hess_f[i, j] = (  f(x + E[:, i] + E[:, j]) 
                            - f(x - E[:, i] + E[:, j]) 
                            - f(x + E[:, i] - E[:, j]) 
                            + f(x - E[:, i] - E[:, j]) ) * (0.25 / (E[i, i] * E[j, j]))
    
    return hess_f

In [133]:
def coor_descent(f, x0, tol=1e-5, maxiter=100):
    '''
        Input:
            f - func to minimize
            x0 - initial point
            tol - tolerance
            maxiter - max num of iterations
        Output:
            xf - final aproximation of x*
            iterations - number of iterations to reach soln
    '''
    
    xf = x0
    n = len(x0)
    iterations = 0
    c_1 = 0.1
    
    d = np.zeros(n)
    grad_f = grad(f, xf)
    
    while iterations < maxiter and np.linalg.norm(grad_f, np.inf) > tol:
        
        
        
        #Find descent direction
        i = np.argmax(abs(grad_f))
        d[i] = -np.sign(grad_f[i])
        
        #Direction coef.
        alfa = 1
        
        while f(xf + alfa*d) > f(xf) - alfa*c_1*grad_f[i]:
            alfa /= 2
        
        
        #Next iteration
        xf = xf + alfa*d
        iterations += 1
        d[i] = 0
        grad_f = grad(f, xf)
    
    return xf, iterations
    

In [153]:
def max_descent(f, x0, tol=1e-5, maxiter=100):
    '''
        Input:
            f - func to minimize
            x0 - initial point
            tol - tolerance
            maxiter - max num of iterations
        Output:
            xf - final aproximation of x*
            iterations - number of iterations to reach soln
    '''
    
    xf = x0
    n = len(x0)
    iterations = 0
    c_1 = 0.1
    
    grad_f = grad(f, xf)
    
    while iterations < maxiter and np.linalg.norm(grad_f, np.inf) > tol:
        
        #Find descent direction
        d = - grad_f / np.linalg.norm(grad_f)
        
        #Direction coef.
        alfa = 1
        
        while f(xf + alfa*d) > f(xf) + alfa*c_1*np.dot(grad_f, d):
            alfa /= 2
        
        #Next iteration
        xf = xf + alfa*d
        iterations += 1
        grad_f = grad(f, xf)
    
    return xf, iterations

In [154]:
def newton_descent(f, x0, tol=1e-5, maxiter=100):
    '''
        Input:
            f - func to minimize
            x0 - initial point
            tol - tolerance
            maxiter - max num of iterations
        Output:
            xf - final aproximation of x*
            iterations - number of iterations to reach soln
    '''
    
    xf = x0
    n = len(x0)
    iterations = 0
    c_1 = 0.1
    
    grad_f = grad(f, xf)
    hess_f = hess(f, xf)
    while iterations < maxiter and np.linalg.norm(grad_f, np.inf) > tol:
        
        #Find descent direction
        d = np.linalg.solve(hess_f, -grad_f)
        
        #Direction coef.
        alfa = 1
        
        while f(xf + alfa*d) > f(xf) + alfa*c_1*np.dot(grad_f, d):
            alfa /= 2
        
        #Next iteration
        xf = xf + alfa*d
        iterations += 1
        grad_f = grad(f, xf)
        hess_f = hess(f, xf)
    
    return xf, iterations

In [155]:
'''
    Ejer 2.2.1
'''
A = la.pascal(4)
b = -np.ones(4)
f = lambda x : 1/2 * np.dot(x, np.dot(A, x)) + np.dot(b, x) + 1
x0 = np.array([4, 4, 4, 4])

In [156]:
coor_descent(f, x0, 1e-5, 2000)

(array([  1.00012207e+00,  -2.74658203e-04,   2.25067139e-04,
         -6.38961792e-05]), 1615)

In [157]:
max_descent(f, x0, 1e-5, 2000)

(array([  1.00006423e+00,  -1.50459317e-04,   1.23717900e-04,
         -3.50451830e-05]), 553)

In [158]:
newton_descent(f, x0, 1e-5, 2000)

(array([  9.99999999e-01,   2.25245600e-09,  -1.50217971e-09,
          3.75111942e-10]), 1)

In [168]:
'''
    Ejer 2.2.2
'''
rosenbrock = lambda x : 100*(x[0]**2 - x[1])**2 + (x[0]-1)**2
x0_r = np.array([2, 3])

In [169]:
coor_descent(rosenbrock, x0_r, 1e-5, 1000)

(array([ 1.46264648,  2.140625  ]), 1000)

In [170]:
max_descent(rosenbrock, x0_r, 1e-5, 1000)

(array([ 1.66059885,  2.75876728]), 1000)

In [172]:
newton_descent(rosenbrock, x0_r, 1e-5, 1000)

(array([ 1.,  1.]), 14)